In [1]:
import bz2
import argparse
from tqdm import tqdm
import math

# Convert scaffolds with gaps labeled as 'N' to list of contigs
def scaffold_to_contigs(seq, min=100, max=4096):
    contigs = seq.split('N')
    # Break configs over max into sub-contigs, with redundancies
    in_range = []
    for contig in contigs:
        if len(contig) > max:
            in_range += [contig[i:i+max] for i in range(0, len(contig), max//2)]
        else:
            in_range.append(contig)
    # Return (sub-)contigs over min
    return [contig for contig in in_range if len(contig) > min]
            

In [2]:
import os
import glob

dir = './data-scratch'
outdir = './data-tmp'

files = sorted(glob.glob(os.path.join(dir, '*.bz2')))

idx = 0

for file in files:
    print("File: ", file)
    with bz2.open(file, 'rt') as f: # read as text
        fasta = f.readlines()

    # Extract sequences from lines, then lines from sequence
    name = ''
    seq = ''
    for line in fasta[0:500]:
        if line[0] == '>':
            # If seq is not empty, we have found the end of a sequence; extract contigs and write to files
            if seq != '':
                contigs = scaffold_to_contigs(seq)

                for contig in contigs:
                    # Generate filename, write as .txt, iterate idx
                    filename = str(idx) + '_' + os.path.basename(file) + '_' + name + '.txt'
                    with open(os.path.join(outdir, filename), 'w') as f:
                        f.write(contig)
                    
                    idx += 1

            # Update name
            name = line[1:-1]
        else:
            seq += line.strip(' \n')

File:  ./data-scratch\SRS022713.scaffolds.fa.bz2
File:  ./data-scratch\SRS042628.scaffolds.fa.bz2
